In [1]:
import os
import shutil
import numpy as np
import efficientnet.keras as efn

from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
# from tensorflow.keras.optimizers.legacy import RMSprop
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2024-07-13 07:05:36.434066: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-13 07:05:36.457632: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-13 07:05:36.457665: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-13 07:05:36.471615: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
import wandb
run = wandb.init()
artifact = run.use_artifact('malitkalha370/Maram-Lagophthalmos/Training:v1', type='dataset')
artifact_dir = artifact.download('./')

In [2]:
# Define the augmentation configuration for training
train_datagen = ImageDataGenerator(
    rescale=1./255.,
    horizontal_flip=True,
    validation_split=0.2  # Use 20% of the data for validation
)

# Directory containing the dataset
dataset_dir = 'dataset/'

# Create generators for training and validation
train_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),  # Resize images to a fixed size
    batch_size=32,
    class_mode='binary',  # Use 'categorical' if you have more than 2 classes
    subset='training'  # Use the training subset
)
validation_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),  # Resize images to a fixed size
    batch_size=32,
    class_mode='binary',  # Use 'categorical' if you have more than 2 classes
    subset='validation'  # Use the validation subset
)

Found 170 images belonging to 2 classes.
Found 41 images belonging to 2 classes.


In [3]:
base_model = efn.EfficientNetB0(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)

# Add a final sigmoid layer with 1 node for classification output
predictions = Dense(1, activation="sigmoid")(x)
model_final = Model(inputs = base_model.input, outputs = predictions)

2024-07-13 07:05:48.481498: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10364 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:05:00.0, compute capability: 8.6


In [4]:
model_final.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_bn  │ (None, 112, 112,  │         64 │ block1a_project_… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_conv │ (None, 112, 112,  │      1,536 │ block1a_project_… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_bn   │ (None, 112, 112,  │        384 │ block2a_expand_c… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_act… │ (None, 112, 112,  │          0 │ block2a_expand_b

 Total params: 68,276,893 (260.46 MB)

 Trainable params: 64,227,329 (245.01 MB)

 Non-trainable params: 4,049,564 (15.45 MB)

                                                                                                  
 dense_1 (Dense)             (None, 1)                    1025      ['dropout[0][0]']             
                                                                                                  
Total params: 68276893 (260.46 MB)
Trainable params: 64227329 (245.01 MB)
Non-trainable params: 4049564 (15.45 MB)
__________________________________________________________________________________________________


In [5]:
model_final.compile(optimizer=RMSprop(learning_rate=0.0001, decay=1e-6), loss='binary_crossentropy', metrics=['accuracy'])
# Train the model using fit instead of fit_generator
eff_history = model_final.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=20
)

/usr/local/lib/python3.11/dist-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Epoch 1/20


AttributeError: Exception encountered when calling Activation.call().

[1mmodule 'keras.api.backend' has no attribute 'sigmoid'[0m

Arguments received by Activation.call():
  • inputs=tf.Tensor(shape=(None, 112, 112, 32), dtype=float32)

In [ ]:
# Save the entire model to a HDF5 file
model_final.save('keras_model.h5')


In [ ]:
from tensorflow.keras.models import load_model

# Load the model
model_final_loaded = load_model('keras_model.h5')

In [ ]:
from matplotlib import pyplot as plt
plt.plot(eff_history.history['accuracy'])
plt.plot(eff_history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

# Evaluation

In [ ]:
evaluation_score = model_final.evaluate(validation_generator)

# Print the evaluation results
print(f'Validation Loss: {evaluation_score[0]}')
print(f'Validation Accuracy: {evaluation_score[1]}')

# Inference

In [ ]:
# Function to preprocess the image
def preprocess_image(img_path, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size)  # Load image with target size
    img_array = image.img_to_array(img)  # Convert image to numpy array
    a = img_array / 255.0
    plt.imshow(a)
    plt.show()
    img_array = np.expand_dims(img_array, axis=0)  # Expand dimensions to match model input
    img_array /= 255.0  # Rescale image
    return img_array

# Function to run inference
def run_inference(model, img_path):
    img_array = preprocess_image(img_path)
    prediction = model.predict(img_array)  # Run model prediction
    return prediction, img_array


In [ ]:
# Define the main folder and subfolders
main_folder = 'testing'
subfolders = ['lagophthalmos', 'normal']
files = []
# Loop through each subfolder
for subfolder in subfolders:
    subfolder_path = os.path.join(main_folder, subfolder)
    
    # Check if the subfolder exists
    if os.path.exists(subfolder_path) and os.path.isdir(subfolder_path):
        
        # Loop through each file in the subfolder
        for file_name in os.listdir(subfolder_path):
            source = os.path.join(subfolder_path, file_name)
            files.append(source)
    else:
        print(f"Subfolder {subfolder} does not exist or is not a directory.")

print("Merging complete.")
files

In [ ]:
len(files)

In [ ]:
# Example usage
y_pred = []
results_with_images = {}
for i, f in enumerate(files):
    results_with_images[i] = []
    img_path = f # Path to your image
    print('Image Path' , img_path)
    prediction, img_array = run_inference(model_final, img_path)
    print(i)
    # Interpret the prediction
    if prediction[0][0] < 0.5:
        results_with_images[i].append([img_array, '0'])

        y_pred.extend([0])
        print('Predicted Class: Lagophthalmos')
    else:
        results_with_images[i].append([img_array, '1'])
        y_pred.extend([1])
        print('Predicted Class: Normal')

    print(f'Prediction Confidence: {prediction[0][0]}')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, ConfusionMatrixDisplay
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

model = model_final
# Load your test data
# Make sure to update the directory, target_size, and other parameters according to your dataset
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    'testing',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

# Get the true labels
y_true = test_generator.classes
print('y_pred ', y_pred)
print('y_true ', y_true)

# Compute confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Compute other metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

# Print the metrics
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

# Display the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=test_generator.class_indices.keys())
disp.plot(cmap=plt.cm.Blues)
plt.show()


# False Positives 

In [ ]:
false_positives = []
false_negatives = []
for i, (true, pred) in enumerate(zip(y_true, y_pred)):
    if true == 0 and pred == 1:
        false_positives.append(i)
    elif true == 1 and pred == 0:
        false_negatives.append(i)

In [ ]:
for fp in false_positives:
    print(fp)
    img_array = results_with_images[fp][0][0]
    print('Label: ', label)
    plt.imshow(img_array[0])
    plt.show()

# False Negatives

In [ ]:
for fn in false_negatives:
    print(fn)
    img_array = results_with_images[fn][0][0]
    label = results_with_images[fn][0][1]
    print('Label: ', label)
    plt.imshow(img_array[0])
    plt.show()

# Artifact

In [ ]:
import wandb

# Initialize a new run
wandb.init(project='Maram-Lagophthalmos', entity='malitkalha370')

# Create an artifact
artifact = wandb.Artifact(name='Models', type='model')

# Add files to the artifact
artifact.add_file('./keras_model.h5')

# Log the artifact
wandb.log_artifact(artifact)

# Finish the run
wandb.finish()
